
# Tissue Detection for WSIs

To patching or tiling, We need to seperate specimans from background in WSIs(Whole Slide Images).

## Prerequisites
`
brew install openslide
`  
`
pip3 install -U matplotlib numpy openslide-python Pillow scikit-image scikit-learn scipy
`

In [1]:
from openslide import OpenSlide, OpenSlideError

In [ ]:
path = 'test_images/TEST_CERVIX_001.svs'
osr = OpenSlide(path)

In [ ]:
osr.get_thumbnail((1024, 512))

In [2]:
import cv2
import numpy as np

def scaleContourDim(contours, scale):
    return [np.array(cont * scale, dtype='int32') for cont in contours]


def scaleHolesDim(contours, scale):
    return [[np.array(hole * scale, dtype = 'int32') for hole in holes] for holes in contours]
 

def segmentTissue(wsi_img, seg_level=0, sthresh=20, sthresh_up = 255, mthresh=7, close = 0, use_otsu=True, 
                  filter_params={'a_t':100, }, ref_patch_size=512,
                  exclude_ids=[], keep_ids=[]):
    """
        Segment the tissue via HSV -> Median thresholding -> Binary threshold
    """
    def _assertLevelDownsamples(wsi_img):
        level_downsamples = []
        dim_0 = wsi_img.level_dimensions[0]

        for downsample, dim in zip(wsi_img.level_downsamples, wsi_img.level_dimensions):
            estimated_downsample = (dim_0[0]/float(dim[0]), dim_0[1]/float(dim[1]))
            level_downsamples.append(estimated_downsample) if estimated_downsample != (downsample, downsample) else level_downsamples.append((downsample, downsample))

        return level_downsamples



    def _filter_contours(contours, hierarchy, filter_params):
        """
            Filter contours by: area.
        """
        filtered = []

        # find indices of foreground contours (parent == -1)
        hierarchy_1 = np.flatnonzero(hierarchy[:,1] == -1)
        all_holes = []

        # loop through foreground contour indices
        for cont_idx in hierarchy_1:
            # actual contour
            cont = contours[cont_idx]
            # indices of holes contained in this contour (children of parent contour)
            holes = np.flatnonzero(hierarchy[:, 1] == cont_idx)
            # take contour area (includes holes)
            a = cv2.contourArea(cont)
            # calculate the contour area of each hole
            hole_areas = [cv2.contourArea(contours[hole_idx]) for hole_idx in holes]
            # actual area of foreground contour region
            a = a - np.array(hole_areas).sum()
            if a == 0: continue
            if tuple((filter_params['a_t'],)) < tuple((a,)): 
                filtered.append(cont_idx)
                all_holes.append(holes)


        foreground_contours = [contours[cont_idx] for cont_idx in filtered]

        hole_contours = []

        for hole_ids in all_holes:
            unfiltered_holes = [contours[idx] for idx in hole_ids ]
            unfilered_holes = sorted(unfiltered_holes, key=cv2.contourArea, reverse=True)
            # take max_n_holes largest holes by area
            unfilered_holes = unfilered_holes[:filter_params['max_n_holes']]
            filtered_holes = []

            # filter these holes
            for hole in unfilered_holes:
                if cv2.contourArea(hole) > filter_params['a_h']:
                    filtered_holes.append(hole)

            hole_contours.append(filtered_holes)

        return foreground_contours, hole_contours

    img = np.array(wsi_img.read_region((0,0), seg_level, wsi_img.level_dimensions[seg_level]))
    img_hsv = cv2.cvtColor(img, cv2.COLOR_RGB2HSV)  # Convert to HSV space
    img_med = cv2.medianBlur(img_hsv[:,:,1], mthresh)  # Apply median blurring


    # Thresholding
    if use_otsu:
        _, img_otsu = cv2.threshold(img_med, 0, sthresh_up, cv2.THRESH_OTSU+cv2.THRESH_BINARY)
    else:
        _, img_otsu = cv2.threshold(img_med, sthresh, sthresh_up, cv2.THRESH_BINARY)

    # Morphological closing
    if close > 0:
        kernel = np.ones((close, close), np.uint8)
        img_otsu = cv2.morphologyEx(img_otsu, cv2.MORPH_CLOSE, kernel)                 
    
    level_downsamples = _assertLevelDownsamples(wsi_img)
    scale = level_downsamples[seg_level]
    scaled_ref_patch_area = int(ref_patch_size**2 / (scale[0] * scale[1]))
    filter_params = filter_params.copy()
    filter_params['a_t'] = filter_params['a_t'] * scaled_ref_patch_area
    filter_params['a_h'] = filter_params['a_h'] * scaled_ref_patch_area

    # Find and filter contours
    contours, hierarchy = cv2.findContours(img_otsu, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_NONE) # Find contours 
    hierarchy = np.squeeze(hierarchy, axis=(0,))[:, 2:]
    if filter_params: foreground_contours, hole_contours = _filter_contours(contours, hierarchy, filter_params)  # Necessary for filtering out artifacts

    contours_tissue = scaleContourDim(foreground_contours, scale)
    holes_tissue = scaleHolesDim(hole_contours, scale)

    #exclude_ids = [0,7,9]
    if len(keep_ids) > 0:
        contour_ids = set(keep_ids) - set(exclude_ids)
    else:
        contour_ids = set(np.arange(len(contours_tissue))) - set(exclude_ids)

    contours_tissue = [contours_tissue[i] for i in contour_ids]
    holes_tissue = [holes_tissue[i] for i in contour_ids]
    
    return contours_tissue, holes_tissue

In [ ]:
# filter_params description
# a_t: area_threshold(pass the more value)
# a_h: hole_threshold(pass the lower value)
ct, ht = segmentTissue(osr, filter_params={'a_t':10, 'a_h': 16, 'max_n_holes':5 })

In [ ]:
len(ct)

In [ ]:
x, y = osr.dimensions
target_x = 512
target_y = int(y/(x/target_x))
divisor = (x/target_x, y/target_y)
cnt = []
for i in ct:
    t = np.floor_divide(i, divisor)
    x = np.array(t, dtype='int32')
    cnt.append(x)

In [ ]:
import matplotlib.pylab as plt

numpy_image=np.array(osr.get_thumbnail((512, 105)))  

# convert to a openCV2 image and convert from RGB to BGR format
opencv_image=cv2.cvtColor(numpy_image, cv2.COLOR_RGB2BGR)

cv2.drawContours(opencv_image, cnt, -1, (0, 255, 0), 1)
plt.figure(figsize = (512,105))
plt.imshow(opencv_image)

Patching process

In [3]:
# other imports
import cv2

class Contour_Checking_fn(object):
    # Defining __call__ method 
    def __call__(self, pt): 
        raise NotImplementedError

class isInContourV1(Contour_Checking_fn):
    def __init__(self, contour):
        self.cont = contour

    def __call__(self, pt): 
        return 1 if cv2.pointPolygonTest(self.cont, pt, False) >= 0 else 0

class isInContourV2(Contour_Checking_fn):
    def __init__(self, contour, patch_size):
        self.cont = contour
        self.patch_size = patch_size

    def __call__(self, pt): 
        return 1 if cv2.pointPolygonTest(self.cont, (pt[0]+self.patch_size//2, pt[1]+self.patch_size//2), False) >= 0 else 0

# Easy version of 4pt contour checking function - 1 of 4 points need to be in the contour for test to pass
class isInContourV3_Easy(Contour_Checking_fn):
    def __init__(self, contour, patch_size, center_shift=0.5):
        self.cont = contour
        self.patch_size = patch_size
        self.shift = int(patch_size//2*center_shift)
    def __call__(self, pt): 
        center = (pt[0]+self.patch_size//2, pt[1]+self.patch_size//2)
        if self.shift > 0:
            all_points = [(center[0]-self.shift, center[1]-self.shift),
                          (center[0]+self.shift, center[1]+self.shift),
                          (center[0]+self.shift, center[1]-self.shift),
                          (center[0]-self.shift, center[1]+self.shift)
                          ]
        else:
            all_points = [center]

        for points in all_points:
            if cv2.pointPolygonTest(self.cont, points, False) >= 0:
                return 1
        return 0

# Hard version of 4pt contour checking function - all 4 points need to be in the contour for test to pass
class isInContourV3_Hard(Contour_Checking_fn):
    def __init__(self, contour, patch_size, center_shift=0.5):
        self.cont = contour
        self.patch_size = patch_size
        self.shift = int(patch_size//2*center_shift)
    def __call__(self, pt): 
        center = (pt[0]+self.patch_size//2, pt[1]+self.patch_size//2)
        if self.shift > 0:
            all_points = [(center[0]-self.shift, center[1]-self.shift),
                          (center[0]+self.shift, center[1]+self.shift),
                          (center[0]+self.shift, center[1]-self.shift),
                          (center[0]-self.shift, center[1]+self.shift)
                          ]
        else:
            all_points = [center]

        for points in all_points:
            if cv2.pointPolygonTest(self.cont, points, False) < 0:
                return 0
        return 1

In [4]:
def isWhitePatch(patch, satThresh=5):
    patch_hsv = cv2.cvtColor(patch, cv2.COLOR_RGB2HSV)
    return True if np.mean(patch_hsv[:,:,1]) < satThresh else False

def isBlackPatch(patch, rgbThresh=40):
    return True if np.all(np.mean(patch, axis = (0,1)) < rgbThresh) else False


In [5]:
def isInHoles(holes, pt, patch_size):
    for hole in holes:
        if cv2.pointPolygonTest(hole, (pt[0]+patch_size/2, pt[1]+patch_size/2), False) > 0:
            return 1

    return 0


def isInContours(cont_check_fn, pt, holes=None, patch_size=256):
    if cont_check_fn(pt):
        if holes is not None:
            return not isInHoles(holes, pt, patch_size)
        else:
            return 1
    return 0


def _getPatchGenerator(wsi_img, cont, cont_idx, ht, patch_level, save_path, patch_size=256, step_size=256, custom_downsample=1,
    white_black=True, white_thresh=15, black_thresh=50, contour_fn='four_pt', use_padding=True):
    start_x, start_y, w, h = cv2.boundingRect(cont) if cont is not None else (0, 0, wsi_img.level_dimensions[patch_level][0], wsi_img.level_dimensions[patch_level][1])
    print("Bounding Box:", start_x, start_y, w, h)
    print("Contour Area:", cv2.contourArea(cont))

    
    def _assertLevelDownsamples(wsi_img):
        level_downsamples = []
        dim_0 = wsi_img.level_dimensions[0]

        for downsample, dim in zip(wsi_img.level_downsamples, wsi_img.level_dimensions):
            estimated_downsample = (dim_0[0]/float(dim[0]), dim_0[1]/float(dim[1]))
            level_downsamples.append(estimated_downsample) if estimated_downsample != (downsample, downsample) else level_downsamples.append((downsample, downsample))

        return level_downsamples


    if custom_downsample > 1:
        assert custom_downsample == 2 
        target_patch_size = patch_size
        patch_size = target_patch_size * 2
        step_size = step_size * 2
        print("Custom Downsample: {}, Patching at {} x {}, But Final Patch Size is {} x {}".format(custom_downsample, patch_size, patch_size, 
            target_patch_size, target_patch_size))
    
    level_downsamples = _assertLevelDownsamples(wsi_img)
    patch_downsample = (int(level_downsamples[patch_level][0]), int(level_downsamples[patch_level][1]))
    ref_patch_size = (patch_size*patch_downsample[0], patch_size*patch_downsample[1])

    step_size_x = step_size * patch_downsample[0]
    step_size_y = step_size * patch_downsample[1]

    if isinstance(contour_fn, str):
        if contour_fn == 'four_pt':
            cont_check_fn = isInContourV3_Easy(contour=cont, patch_size=ref_patch_size[0], center_shift=0.5)
        elif contour_fn == 'four_pt_hard':
            cont_check_fn = isInContourV3_Hard(contour=cont, patch_size=ref_patch_size[0], center_shift=0.5)
        elif contour_fn == 'center':
            cont_check_fn = isInContourV2(contour=cont, patch_size=ref_patch_size[0])
        elif contour_fn == 'basic':
            cont_check_fn = isInContourV1(contour=cont)
        else:
            raise NotImplementedError
    else:
        assert isinstance(contour_fn, Contour_Checking_fn)
        cont_check_fn = contour_fn

    img_w, img_h = wsi_img.level_dimensions[0]
    if use_padding:
        stop_y = start_y+h
        stop_x = start_x+w
    else:
        stop_y = min(start_y+h, img_h-ref_patch_size[1])
        stop_x = min(start_x+w, img_w-ref_patch_size[0])

    count = 0
    for y in range(start_y, stop_y, step_size_y):
        for x in range(start_x, stop_x, step_size_x):

            if not isInContours(cont_check_fn, (x,y), ht[cont_idx], ref_patch_size[0]): #point not inside contour and its associated holes
                continue    

            count+=1
            patch_PIL = wsi_img.read_region((x,y), patch_level, (patch_size, patch_size)).convert('RGB')
            if custom_downsample > 1:
                patch_PIL = patch_PIL.resize((target_patch_size, target_patch_size))

            if white_black:
                if isBlackPatch(np.array(patch_PIL), rgbThresh=black_thresh) or isWhitePatch(np.array(patch_PIL), satThresh=white_thresh): 
                    continue

            patch_info = {'x':x // (patch_downsample[0] * custom_downsample), 
                          'y':y // (patch_downsample[1] * custom_downsample), 
                          'cont_idx':cont_idx, 'patch_level':patch_level, 
                          'downsample': level_downsamples[patch_level], 
                          'downsampled_level_dim': tuple(np.array(wsi_img.level_dimensions[patch_level])//custom_downsample), 
                          'level_dim': wsi_img.level_dimensions[patch_level],
                          'patch_PIL':patch_PIL, 
                          #'name':self.name, 
                          'save_path':save_path}

            yield patch_info


    print("patches extracted: {}".format(count))

In [ ]:
'''Patching params description
custom_downsample: 1 or 2
'''
from PIL import Image
patching_param = {'patch_level': 0, 
                  'patch_size': 512, 
                  'step_size': 512, 
                  'save_path': 'test_output', 
                  'custom_downsample': 1}
_counter = 0
for idx, cont in enumerate(ct):
    patch_gen = _getPatchGenerator(osr, cont, idx, ht, contour_fn='four_pt', **patching_param)
#     patch_gen = _getPatchGenerator(osr, cont, idx, ht, contour_fn='four_pt_hard', **patching_param)

    try:
        first_patch = next(patch_gen)

    # empty contour, continue
    except StopIteration:
        print(f'StopIteration Error')
        continue

    for patch in patch_gen:
        patch['patch_PIL'].save(f"{patch['save_path']}/TestPatch_{_counter}.png", "png")
#         Image.save(patch['save_path'] + "TestPatch" + _counter, "JPEG")
        _counter += 1        
#         with Image.open(patch['patch_PIL']) as im:
#             im.thumbnail(size)
#             im.save(patch['save_path'] + "TestPatch", _counter, "JPEG")
#             _couter += 1

In [6]:
from os import walk

def get_list(dir_) -> list:
    list_ = []
#     for (dirpath, dirnames, filenames) in walk(dir_):
    for (_, _, filenames) in walk(dir_):
        list_ = filenames
    return list_

In [13]:
dataset_path = '/data/kwkim/aadw/dataset/bladder/raw_test'

In [14]:
list_ = get_list(dataset_path)

In [15]:
len(list_)

23

In [16]:
filter_params={'a_t':10, 'a_h': 16, 'max_n_holes':5 }
patching_param = {'patch_level': 0, 
                  'patch_size': 1024, 
                  'step_size': 1024, 
                  'save_path': '/data/kwkim/aadw/dataset/bladder/patches-test-1024', 
                  'custom_downsample': 1}

In [17]:
import os
if os.path.exists(patching_param['save_path']) is False:
    os.mkdir(patching_param['save_path'])
    
for i in list_:
    try:
        os.mkdir(os.path.join(patching_param['save_path'], os.path.splitext(i)[0]))
    except:
        continue

In [18]:
%%time
import time
from tqdm import tqdm
for file_ in tqdm(list_):
    try:
        osr = OpenSlide(os.path.join(dataset_path, file_))
    except:
        print(f'Could not open: {file_}')
        continue
    
    start_t = time.time()
    ct, ht = segmentTissue(osr, filter_params=filter_params)    

    end_t = time.time()
    print(f'[D] Elapsed(Tissue detection): {end_t-start_t}s')
    filename = os.path.basename(file_).split('.')[0]
#     if os.path.exists(os.join.path(patching_param['save_path'], filename)) is False:
#         os.mkdir(os.join.path(patching_param['save_path'], filename))
    start_t = time.time()
    
    for idx, cont in enumerate(ct):
#         patch_gen = _getPatchGenerator(osr, cont, idx, ht, contour_fn='four_pt', **patching_param)
        patch_gen = _getPatchGenerator(osr, cont, idx, ht, contour_fn='four_pt_hard', **patching_param)

        try:
            first_patch = next(patch_gen)

        # empty contour, continue
        except StopIteration:
            print(f'StopIteration Error')
            continue
        
        for patch in patch_gen:
            patch['patch_PIL'].save(f"{patch['save_path']}/{filename}/{filename}_x_{patch['x']}_y_{patch['y']}.png", "png")
    end_t = time.time()
    print(f'[D] Elapsed(Generate patches): {end_t-start_t}s')
    osr.close()

  0%|                                                                                                       | 0/23 [00:00<?, ?it/s]

[D] Elapsed(Tissue detection): 102.32381105422974s
Bounding Box: 8403 22953 6239 5904
Contour Area: 17303038.0
patches extracted: 12
Bounding Box: 22333 21998 2576 4525
Contour Area: 5192546.5
patches extracted: 5
Bounding Box: 14733 21826 6214 6548
Contour Area: 23390904.0
patches extracted: 16
Bounding Box: 24464 18975 5367 3929
Contour Area: 8215303.5
patches extracted: 3
Bounding Box: 12272 15121 9244 7016
Contour Area: 41226878.0
patches extracted: 30
Bounding Box: 22083 11974 5914 8107
Contour Area: 31563113.5
patches extracted: 21
Bounding Box: 5839 10383 7728 9179
Contour Area: 29799544.0
patches extracted: 22
Bounding Box: 4805 6359 2959 3848
Contour Area: 6323984.0
patches extracted: 3
Bounding Box: 12764 6215 7231 9739
Contour Area: 35945338.0
patches extracted: 23
Bounding Box: 3378 5010 9386 10524
Contour Area: 35599955.0
patches extracted: 23
Bounding Box: 11725 3470 5722 3493
Contour Area: 10004928.5
patches extracted: 6
Bounding Box: 19768 416 6103 6133
Contour Area: 16

  4%|████                                                                                        | 1/23 [02:44<1:00:19, 164.54s/it]

patches extracted: 6
[D] Elapsed(Generate patches): 62.139723777770996s
[D] Elapsed(Tissue detection): 1463.3163831233978s
Bounding Box: 43823 32315 2563 7329
Contour Area: 4310393.5
patches extracted: 0
StopIteration Error
Bounding Box: 21506 32130 10476 7633
Contour Area: 37783144.0
patches extracted: 23
Bounding Box: 31706 31628 5483 3417
Contour Area: 12944329.0
patches extracted: 9
Bounding Box: 6397 31146 4040 10453
Contour Area: 23176432.5
patches extracted: 17
Bounding Box: 40480 30241 5209 4382
Contour Area: 9701354.5
patches extracted: 3
Bounding Box: 15834 29144 3572 2916
Contour Area: 4570024.5
patches extracted: 1
Bounding Box: 5924 28281 3772 4999
Contour Area: 7179322.0
patches extracted: 4
Bounding Box: 21260 27835 4678 3160
Contour Area: 8663475.5
patches extracted: 5
Bounding Box: 47490 24720 14884 11568
Contour Area: 51124366.5
patches extracted: 29
Bounding Box: 9576 24535 3779 3742
Contour Area: 5389616.0
patches extracted: 2
Bounding Box: 28169 24140 12124 7527
Co

  9%|███████▉                                                                                   | 2/23 [30:01<6:00:44, 1030.69s/it]

...

patches extracted: 7
[D] Elapsed(Generate patches): 182.508807182312s
[D] Elapsed(Tissue detection): 755.0467712879181s
Bounding Box: 40381 41154 4186 4146
Contour Area: 9148433.5
patches extracted: 4
Bounding Box: 10791 34882 8943 7383
Contour Area: 41452070.0
patches extracted: 31
Bounding Box: 6253 34129 2773 3063
Contour Area: 4118782.0
patches extracted: 2
Bounding Box: 22218 32062 7098 7242
Contour Area: 25419880.5
patches extracted: 16
Bounding Box: 53736 29161 4357 5373
Contour Area: 9349941.0
patches extracted: 3
Bounding Box: 45666 25559 8758 14765
Contour Area: 73582106.5
patches extracted: 50
Bounding Box: 8408 25393 6433 7540
Contour Area: 23033198.5
patches extracted: 15
Bounding Box: 1063 24517 7637 9530
Contour Area: 42404507.5
patches extracted: 29
Bounding Box: 47170 22073 9043 2879
Contour Area: 13488400.5
patches extracted: 8
Bounding Box: 12273 20510 4347 4921
Contour Area: 9516986.5
patches extracted: 6
Bounding Box: 14659 19237 31257 25270
Contour Area: 350435835

 96%|███████████████████████████████████████████████████████████████████████████████████████    | 22/23 [6:52:47<15:29, 929.24s/it]

patches extracted: 34
[D] Elapsed(Generate patches): 312.52991127967834s
[D] Elapsed(Tissue detection): 356.36781668663025s
Bounding Box: 11990 34613 3791 3821
Contour Area: 5367006.5
patches extracted: 1
Bounding Box: 30906 33052 6033 3499
Contour Area: 10261898.5
patches extracted: 4
Bounding Box: 27245 30246 3160 3887
Contour Area: 5966048.5
patches extracted: 1
Bounding Box: 24044 30202 21970 11345
Contour Area: 68353368.0
patches extracted: 47
Bounding Box: 3974 29976 7370 7571
Contour Area: 12393151.0
patches extracted: 4
Bounding Box: 29066 28634 6659 3570
Contour Area: 5456813.5
patches extracted: 1
Bounding Box: 16981 28035 3320 5583
Contour Area: 7276438.0
patches extracted: 3
Bounding Box: 7906 27982 4271 6436
Contour Area: 15222194.5
patches extracted: 8
Bounding Box: 12164 27171 4538 8290
Contour Area: 8019693.5
patches extracted: 1
Bounding Box: 18798 25766 5958 10322
Contour Area: 25287484.5
patches extracted: 12
Bounding Box: 6485 25446 2125 3380
Contour Area: 3866508.0

100%|██████████████████████████████████████████████████████████████████████████████████████████| 23/23 [7:01:17<00:00, 1099.04s/it]

patches extracted: 12
[D] Elapsed(Generate patches): 153.7496213912964s
CPU times: user 6h 41min 5s, sys: 7min 35s, total: 6h 48min 40s
Wall time: 7h 1min 17s
